In [1]:
# Step 1: Import Required Libraries

import time
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Step 2: Define model location and SageMaker session

model_uri = "s3://sagemaker-us-east-2-998072949015/wine-quality-model/model.tar.gz"
role = get_execution_role()
sagemaker_session = sagemaker.Session()

In [3]:
# Step 3: Specify entry point script for inference
# This must define a 'model_fn' and 'predict_fn' in a file called predictor.py

sklearn_model = SKLearnModel(
    model_data=model_uri,
    role=role,
    entry_point="predictor.py",  # make sure this file is in your working directory
    framework_version="1.2-1",
    sagemaker_session=sagemaker_session
)

In [4]:
# Step 4: Deploy the model to a real-time endpoint

endpoint_name = f"wine-quality-endpoint-{int(time.time())}"
predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)

print(f"Endpoint deployed at: {endpoint_name}")

--------!Endpoint deployed at: wine-quality-endpoint-1748042764


In [5]:
# Step 5: Test the Endpoint

import pandas as pd

# Step 5: Create a sample input (one row of wine data)
# Make sure this matches the order and format of your model's training data
sample_input = pd.DataFrame([{
    "fixed acidity": 7.4,
    "volatile acidity": 0.70,
    "citric acid": 0.00,
    "residual sugar": 1.9,
    "chlorides": 0.076,
    "free sulfur dioxide": 11.0,
    "total sulfur dioxide": 34.0,
    "density": 0.9978,
    "pH": 3.51,
    "sulphates": 0.56,
    "alcohol": 9.4
}])

# Step 6: Run the prediction
prediction = predictor.predict(sample_input)

print("Predicted wine quality score:", prediction)

Predicted wine quality score: [5.04834495]
